<a href="https://colab.research.google.com/github/guialmussa-phd/ClimateCodes/blob/main/Correla%C3%A7%C3%A3o_entre_clima_e_conte%C3%BAdo_relativo_de_%C3%A1gua_(RWC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================================
# Correlação entre clima e conteúdo relativo de água (RWC)
# Exemplo em cafeeiros (dados fictícios)
# Guilherme A. L. Torres
# ========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr, f_oneway

# --------------------------------------------------------
# Dados fictícios: medições semanais de RWC + clima
np.random.seed(42)
datas = pd.date_range("2022-01-01", "2022-12-31", freq="7D")

df = pd.DataFrame({
    "Data": datas,
    "Tmax": np.random.normal(29, 3, len(datas)),   # °C
    "Tmin": np.random.normal(18, 2, len(datas)),   # °C
    "Precipitacao": np.random.gamma(2, 3, len(datas)), # mm
    "RWC": np.random.normal(75, 5, len(datas)) - np.linspace(0, 10, len(datas)) # tendência seca
})

df.set_index("Data", inplace=True)

# --------------------------------------------------------
# Estatísticas descritivas
print("Resumo estatístico:")
print(df.describe())

# --------------------------------------------------------
# Correlações
corr_pearson = df.corr(method="pearson")
corr_spearman = df.corr(method="spearman")

print("\nCorrelação de Pearson com RWC:")
print(corr_pearson["RWC"])

print("\nCorrelação de Spearman com RWC:")
print(corr_spearman["RWC"])

# --------------------------------------------------------
# Visualizações
# 1. Série temporal RWC vs Precipitação
fig, ax1 = plt.subplots(figsize=(12,5))
ax1.plot(df.index, df["RWC"], color="green", marker="o", label="RWC (%)")
ax1.set_ylabel("RWC (%)", color="green")
ax2 = ax1.twinx()
ax2.bar(df.index, df["Precipitacao"], color="blue", alpha=0.4, label="Precipitação (mm)")
ax2.set_ylabel("Precipitação (mm)", color="blue")
plt.title("RWC (%) vs Precipitação semanal")
plt.show()

# 2. Dispersão RWC vs Tmax
plt.figure(figsize=(7,5))
sns.regplot(x="Tmax", y="RWC", data=df, ci=None, scatter_kws={"s":60}, color="red")
plt.title("RWC (%) vs Tmax")
plt.xlabel("Temperatura máxima (°C)")
plt.ylabel("RWC (%)")
plt.show()

# 3. Matriz de correlação (heatmap)
plt.figure(figsize=(7,5))
sns.heatmap(corr_pearson, annot=True, cmap="RdYlGn", fmt=".2f")
plt.title("Matriz de correlação (Pearson)")
plt.show()

# --------------------------------------------------------
# Exemplo de ANOVA (comparação entre grupos de chuva alta vs baixa)
df["Grupo_Chuva"] = pd.qcut(df["Precipitacao"], q=2, labels=["Baixa chuva", "Alta chuva"])
anova = f_oneway(
    df[df["Grupo_Chuva"]=="Baixa chuva"]["RWC"],
    df[df["Grupo_Chuva"]=="Alta chuva"]["RWC"]
)
print("\nResultado da ANOVA RWC entre grupos de chuva:")
print(f"F = {anova.statistic:.3f}, p = {anova.pvalue:.4f}")
